In [1]:
!pip install zstandard
!pip install stockfish
!pip install chess


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import chess
import chess.pgn
import concurrent
from src.data_loading import save_dataset_to_csv
from src.raw_data_gather import gather
from src.patches import GAMES_DATASET_PATCH, GAMES_PATCH, STOCKFISH_PATH
from stockfish import Stockfish
from concurrent.futures import ThreadPoolExecutor

Download data

In [3]:
gather("https://database.lichess.org/standard",
       "lichess_db_standard_rated_2014-05.pgn.zst",
       GAMES_PATCH)

Load games

In [4]:
def load(k: int) -> [chess.pgn.GameNode]:
    pgn = open(GAMES_PATCH, encoding="utf-8")
    result = []

    game = chess.pgn.read_game(pgn)
    count = 0
    while game is not None and count < k:
        if game.variations:
            result.append(game)
        game = chess.pgn.read_game(pgn)
        count += 1

    print("Games count: ", count)
    pgn.close()
    return result

Evaluate positions

In [5]:
def generate_moves_for_games(game: chess.pgn.GameNode) -> [str]:
    result = []
    while game.variations:
        result.append(game.variations[0])
        game = game.variations[0]
    return result

In [6]:
generate_moves_for_games(load(1)[0])

Games count:  1


[<ChildNode at 0x23dc0775340 (1. e4 ...)>,
 <ChildNode at 0x23dc0775400 (1... e5 ...)>,
 <ChildNode at 0x23dc07753d0 (2. Nf3 ...)>,
 <ChildNode at 0x23dc07755e0 (2... f6 ...)>,
 <ChildNode at 0x23dc0775700 (3. d4 ...)>,
 <ChildNode at 0x23dc07758e0 (3... exd4 ...)>,
 <ChildNode at 0x23dc07759a0 (4. Nxd4 ...)>,
 <ChildNode at 0x23dc0775af0 (4... d5 ...)>,
 <ChildNode at 0x23dc0775d00 (5. Bd3 ...)>,
 <ChildNode at 0x23dc0775e20 (5... dxe4 ...)>,
 <ChildNode at 0x23dc0775f40 (6. Bxe4 ...)>,
 <ChildNode at 0x23dc0776090 (6... c5 ...)>,
 <ChildNode at 0x23dc07762a0 (7. O-O ...)>,
 <ChildNode at 0x23dc07763c0 (7... Qxd4 ...)>,
 <ChildNode at 0x23dc0776450 (8. Qxd4 ...)>,
 <ChildNode at 0x23dc0776720 (8... cxd4 ...)>,
 <ChildNode at 0x23dc0776990 (9. Bf4 ...)>,
 <ChildNode at 0x23dc0776d80 (9... Nc6 ...)>,
 <ChildNode at 0x23dc0776c90 (10. Bxc6+ ...)>,
 <ChildNode at 0x23dc0776ed0 (10... bxc6 ...)>,
 <ChildNode at 0x23dc0777110 (11. Nd2 ...)>,
 <ChildNode at 0x23dc0777530 (11... Bb4 ...)>,
 <

In [7]:
def generate_fen_for_moves(moves: [str]) -> [str]:
    result = []
    board = chess.Board()
    for move in moves:
        board.push_uci(move.uci())
        result.append(board.fen())
    return result

In [8]:
generate_fen_for_moves(generate_moves_for_games(load(1)[0]))

Games count:  1


['rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
 'rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2',
 'rnbqkbnr/pppp2pp/5p2/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 0 3',
 'rnbqkbnr/pppp2pp/5p2/4p3/3PP3/5N2/PPP2PPP/RNBQKB1R b KQkq - 0 3',
 'rnbqkbnr/pppp2pp/5p2/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4',
 'rnbqkbnr/pppp2pp/5p2/8/3NP3/8/PPP2PPP/RNBQKB1R b KQkq - 0 4',
 'rnbqkbnr/ppp3pp/5p2/3p4/3NP3/8/PPP2PPP/RNBQKB1R w KQkq - 0 5',
 'rnbqkbnr/ppp3pp/5p2/3p4/3NP3/3B4/PPP2PPP/RNBQK2R b KQkq - 1 5',
 'rnbqkbnr/ppp3pp/5p2/8/3Np3/3B4/PPP2PPP/RNBQK2R w KQkq - 0 6',
 'rnbqkbnr/ppp3pp/5p2/8/3NB3/8/PPP2PPP/RNBQK2R b KQkq - 0 6',
 'rnbqkbnr/pp4pp/5p2/2p5/3NB3/8/PPP2PPP/RNBQK2R w KQkq - 0 7',
 'rnbqkbnr/pp4pp/5p2/2p5/3NB3/8/PPP2PPP/RNBQ1RK1 b kq - 1 7',
 'rnb1kbnr/pp4pp/5p2/2p5/3qB3/8/PPP2PPP/RNBQ1RK1 w kq - 0 8',
 'rnb1kbnr/pp4pp/5p2/2p5/3QB3/8/PPP2PPP/RNB2RK1 b kq - 0 8',
 'rnb1kbnr/pp4pp/5p2/8/3pB3/8/PPP2

In [9]:
def games_to_unique_fens(games: [chess.pgn.GameNode]) -> {str}:
    return {fen
            for game in games
            for fen in generate_fen_for_moves(generate_moves_for_games(game))}

In [10]:
games_to_unique_fens(load(10))

Games count:  10


{'1r1N1rk1/p2bnpbp/4pnp1/2p1p3/4P3/2PP1P2/PP1BB1PP/R2Q1RK1 b - - 0 15',
 '1r1q1rk1/pN1bnpbp/4pnp1/2p1p3/4P3/2PP1P2/PP1BB1PP/R2Q1RK1 w - - 1 15',
 '1r1r2k1/p2bnpbp/4pnp1/2p1p3/4P3/1PPP1P2/P2BB1PP/R2Q1RK1 b - - 0 16',
 '1r1r2k1/p2bnpbp/4pnp1/2p1p3/4P3/2PP1P2/PP1BB1PP/R2Q1RK1 w - - 0 16',
 '1r1r2k1/p3npbp/2b1pnp1/2p1p3/2P1P3/1P1P1P2/P2BB1PP/R2Q1RK1 b - - 0 17',
 '1r1r2k1/p3npbp/2b1pnp1/2p1p3/4P3/1PPP1P2/P2BB1PP/R2Q1RK1 w - - 1 17',
 '1r1r2k1/p3npbp/b3pnp1/2p1p3/2P1P3/PP1P1P2/3BB1PP/1R1Q1RK1 b - - 2 19',
 '1r1r2k1/p3npbp/b3pnp1/2p1p3/2P1P3/PP1P1P2/3BB1PP/R2Q1RK1 w - - 1 19',
 '1r1r2k1/pb2npbp/4pnp1/2p1p3/2P1P3/1P1P1P2/P2BB1PP/R2Q1RK1 w - - 1 18',
 '1r1r2k1/pb2npbp/4pnp1/2p1p3/2P1P3/PP1P1P2/3BB1PP/R2Q1RK1 b - - 0 18',
 '1r6/4nkp1/p2q3p/2p1p3/2P1Qp1P/2N2P2/1r4P1/3BK2R b - - 5 34',
 '1r6/4nkp1/p2q3p/2p1p3/2P2p1P/2N2P2/1rQ3P1/3BK2R w - - 4 34',
 '1r6/4nkp1/p2q3p/2p1p3/2P2p1P/2N2P2/2Q3P1/1r1BK2R b - - 3 33',
 '1r6/4nkp1/p2q3p/2p1p3/2P2p1P/5P2/2Q1N1P1/1r1BK2R w - - 2 33',
 '1r6/4nkp1/p6p/2p1p3/2

In [11]:
def evaluate_fen(fen: str, stockfish_path: str) -> str:
    stockfish = Stockfish(stockfish_path)
    stockfish.set_fen_position(fen)
    return stockfish.get_evaluation()

In [12]:
def evaluate_fens(fens: [str], stockfish_path: str) -> [(str, dict)]:
    with concurrent.futures.ThreadPoolExecutor(10) as executor:
        futures = [(fen, executor.submit(evaluate_fen, fen, stockfish_path)) for fen in fens]
    return [(f, e.result()) for f, e in futures]

Generate dataset with evaluated fens

In [13]:
def generate_dataset(size, stockfish_path) -> [(str, float)]:
    return [(f, e["value"]) for f, e in
            evaluate_fens(games_to_unique_fens(load(size)), stockfish_path) if e["type"] == "cp"]

In [15]:
SIZE = 10 ** 4
dataset = generate_dataset(SIZE, STOCKFISH_PATH)
print("Dataset size: ", len(dataset))

Games count:  10000
Dataset size:  583544


In [16]:
save_dataset_to_csv(dataset, GAMES_DATASET_PATCH)